---

# 📘 Documentation Technique : Persistance OptimaSol

Ce Notebook détaille le fonctionnement de la couche de données du projet OptimaSol. L'architecture est divisée en trois niveaux : la base brute (`BaseDB`), les gestionnaires métiers (`ClientManager`, `DecisionsManager`) et l'interface unifiée (`DBManager`).

---

## 1. La Fondation : `Database` (`base_db.py`)

La classe `Database` est le lien direct avec SQLite. Elle ne contient aucune logique métier, seulement la structure (le schéma) des données.

### Fonctions Clés :

* **`__init__(chemin_db)`** : Initialise le chemin du fichier. Si le dossier parent n'existe pas, elle le crée automatiquement.
* **`connect_db()` / `close_db()`** : Gère l'ouverture et la fermeture sécurisée des connexions pour éviter la corruption du fichier `.db`.
* **Méthodes `create_table_xxx()`** : Chaque table possède sa propre méthode de création. Elles définissent les colonnes, les types (INTEGER, REAL, TEXT) et les contraintes (Primary Keys, Foreign Keys).
* **`create_all_tables()`** : Prépare l'intégralité de la base de données en une seule commande.
* **`inspect_database()`** : Outil de diagnostic essentiel qui affiche le schéma de chaque table et le nombre d'enregistrements présents.

---

## 2. Gestion des Paramètres : `ClientManager` (`client_manager.py`)

C'est ici que l'objet complexe `Client` est traduit pour le stockage.

### Fonctions Clés :

* **`create_client_in_db(client)`** :
* Prend un objet `Client` et le décompose dans les tables : `clients`, `constraints`, `water_heaters`, `prices`, `consignes`, `creneaux_hp`, `plages_interdites`.
* **Transformation JSON** : Elle convertit la matrice de profil de consommation (Numpy) en texte JSON pour le stockage.


* **`reconstitute_client(client_id)`** :
* L'inverse de la création. Elle effectue plusieurs requêtes (SELECT avec `LEFT JOIN`) pour reconstruire l'objet `Client` complet avec tous ses sous-objets.


* **`update_client_in_db(...)`** :
* Permet de mettre à jour uniquement certaines parties du client (ex: changer seulement le planning de chauffe) sans écraser le reste.


* **`delete_client(client_id)`** : Supprime toutes les données liées à un client.
* **`list_all_clients()`** : Retourne la liste de tous les IDs clients présents dans la base.

---

## 3. Gestion des Résultats : `DecisionsManager` (`decision_manager.py`)

Cette classe gère les sorties du solveur : "Quelle puissance appliquer à quel moment ?".

### Fonctions Clés :

* **`create_decision_in_db(client_id, date, puissance)`** : Enregistre une valeur de puissance (Watts) pour un instant T (`datetime`) donné.
* **`get_decisions_for_client(client_id, start_date, end_date)`** : Récupère l'historique des décisions. Cette fonction est conçue pour retourner un **DataFrame Pandas**, facilitant l'affichage de graphiques.
* **`update_decision_in_db(...)`** : Modifie une décision déjà existante pour un créneau horaire précis.
* **`delete_decisions_for_client(client_id)`** : Nettoie l'historique des décisions d'un client sans supprimer son profil.

---

## 4. L'Interface Unifiée : `DBManager` (`main_manager.py`)

Le `DBManager` utilise l'**Héritage Multiple**.

### Pourquoi cette structure ?

En héritant de `ClientManager` et `DecisionsManager`, le `DBManager` devient un guichet unique.

* **Initialisation** : Il appelle les constructeurs des deux classes mères.
* **Usage** : Vous n'avez besoin d'instancier qu'un seul objet `db = DBManager("data.db")` pour accéder à l'intégralité des fonctionnalités (créer un client, lire son profil, enregistrer une décision).

---

## 5. Sécurité et Exceptions (`exceptions_db.py`)

Le système utilise des erreurs personnalisées pour une meilleure robustesse :

* **`DatabaseConnexionError`** : Problème d'accès physique au fichier.
* **`DatabaseIntegrityError`** : Tentative d'insertion de données invalides ou ID déjà existant.
* **`ClientNotFound` / `DecisionNotFound**` : Tentative de lecture d'une donnée inexistante.

---

## Exemple Pratique d'Utilisation